# Assessing RMSF and RMSD of MD simulation

Usage: function plotting will plot rmsd, rmsd histogram, rmsf and PCA projection

In [2]:
from __future__ import print_function
%matplotlib inline
import mdtraj as md
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import pandas as pd
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')
#XRD Ensemble
#28 4NPQ
#18 4HFI

In [3]:
residue_selection = 'resSeq 8 to 316'

In [34]:
traj_notes = ['5NJY_pH70_md1','5NJY_pH70_md2','5NJY_pH70_md3','5NJY_F238L_pH70_md1',
                 '5NJY_F238L_pH70_md2','5NJY_F238L_pH70_md3','5NJY_I233T_pH70_md1',
                  '5NJY_I233T_pH70_md2','5NJY_I233T_pH70_md3','5NJY_F238LI233T_pH70_md1',
                  '5NJY_F238LI233T_pH70_md2','5NJY_F238LI233T_pH70_md3','5NJY_pH46_md1','5NJY_pH46_md2','5NJY_pH46_md3','5NJY_F238L_pH46_md1',
                 '5NJY_F238L_pH46_md2','5NJY_F238L_pH46_md3','5NJY_I233T_pH46_md1',
                  '5NJY_I233T_pH46_md2','5NJY_I233T_pH46_md3','5NJY_F238LI233T_pH46_md1',
                  '5NJY_F238LI233T_pH46_md2','5NJY_F238LI233T_pH46_md3','4HFI_pH46_md1','4HFI_pH46_md2','4HFI_pH46_md3','4HFI_F238L_pH46_md1',
                 '4HFI_F238L_pH46_md2','4HFI_F238L_pH46_md3','4HFI_I233T_pH46_md1',
                  '4HFI_I233T_pH46_md2','4HFI_I233T_pH46_md3','4HFI_F238LI233T_pH46_md1',
                  '4HFI_F238LI233T_pH46_md2','4HFI_F238LI233T_pH46_md3','4NPQ_pH70_md5','4NPQ_pH70_md6','4NPQ_pH70_md7','4NPQ_F238L_pH70_md3',
                 '4NPQ_F238L_pH70_md4','4NPQ_F238L_pH70_md5','4NPQ_I233T_pH70_md3',
                  '4NPQ_I233T_pH70_md4','4NPQ_I233T_pH70_md5','4NPQ_F238LI233T_pH70_md3',
                  '4NPQ_F238LI233T_pH70_md4','4NPQ_F238LI233T_pH70_md5']

In [4]:
def create_md_dataframe():    
    md_data = pd.DataFrame(columns=list(['MD_name','pH','replicate','traj_time']))
    return md_data

In [5]:
def create_metadata(md_data= None):   
    def append_metadata(traj_note,location = '/media/scottzhuang/data/MD/',skip=10,md_data= md_data):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        if skip == 1:
            traj = md.load(location + traj_location,top= location + top_location,stride=10)
        else:
            traj = md.load(location + traj_location,top= location + top_location)
        md_name = traj_note[:traj_note.find('pH')-1]
        pH = traj_note[traj_note.find('pH')+2:traj_note.find('pH')+4]
        md_replicate = traj_note[-1]
        for i in range(0,traj.n_frames):
            md_data.loc[md_data.shape[0]+1] = [md_name,pH,md_replicate,i]

    for traj_note in traj_notes:
        append_metadata(traj_note)

In [59]:
def create_system_notation(md_data = None):
    system_notation = 0
    notation = 0
    location = '/media/scottzhuang/data/MD/'
    skip=10
    notations = []
    increment = 0
    for traj_note in traj_notes:
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top= location + top_location)
        for frame in range(0,traj.n_frames):
            notations.append(notation)
        if increment % 3 == 2:
            notation = notation + 1
        increment = increment + 1 
    md_data['system'] = notations

In [6]:
def create_rmsd_data(md_data= None):
    def append_rmsd_data(traj_note,location = '/media/scottzhuang/data/MD/',ref_name = None, skip=10):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top= location + top_location)
        if ref_name != None:
            ref_location = ("/home/scottzhuang/masterthesis/miscellanies/pdb_ensemble/" + ref_name + ".pdb")
        else:
            ref_location = location + top_location
        ref_traj = md.load(ref_location)
        topology = traj.topology
        if traj.n_atoms != ref_traj.n_atoms:
            traj = traj.atom_slice(topology.select(residue_selection))
        traj.superpose(ref_traj,0)
        rmsd_data.extend(list(md.rmsd(traj, ref_traj)*10))
    rmsd_data = []
    for traj_note in traj_notes:
        append_rmsd_data(traj_note)
    md_data['rmsd']= rmsd_data

In [50]:
def create_cppca_data(md_data = None,residue_selection_1 = "resSeq 13 to 198", residue_selection_2 = "resSeq 198 to 316"):
    def combined_ppca_reduced_cartesian(residue_selection_1 = "resSeq 13 to 198", residue_selection_2 = "resSeq 198 to 316"):
        wholetraj = md.load("/home/scottzhuang/masterthesis/miscellanies/pdb_ensemble/new_ensemble.pdb")
        wholetraj.superpose(wholetraj,28)
        topology = wholetraj.topology
        wholetraj_sliced_ecd = wholetraj.atom_slice(topology.select(residue_selection_1))
        wholetraj_sliced_ecd.superpose(wholetraj_sliced_ecd,28)
        wholetraj_sliced_tmd = wholetraj.atom_slice(topology.select(residue_selection_2))
        wholetraj_sliced_tmd.superpose(wholetraj_sliced_tmd,28)
        ppca_ecd = PCA(n_components=1)
        ppca_tmd = PCA(n_components=1)
        partial_reduced_cartesian = [ppca_ecd.fit_transform(wholetraj_sliced_ecd.xyz.reshape(wholetraj_sliced_ecd.n_frames, wholetraj_sliced_ecd.n_atoms * 3)), ppca_tmd.fit_transform(wholetraj_sliced_tmd.xyz.reshape(wholetraj_sliced_tmd.n_frames, wholetraj_sliced_tmd.n_atoms * 3))]    
        return partial_reduced_cartesian, wholetraj_sliced_ecd, wholetraj_sliced_tmd,ppca_ecd,ppca_tmd
    
    def append_projection_on_combined_ppca_data(traj_note, location = '/media/scottzhuang/data/MD/',skip = 10, residue_selection_1 = "resSeq 13 to 198", residue_selection_2 = "resSeq 198 to 316"):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top= location + top_location)
        topology = traj.topology
        traj_sliced_ecd = traj.atom_slice(topology.select(residue_selection_1))
        traj_sliced_tmd = traj.atom_slice(topology.select(residue_selection_2))

        traj_sliced_ecd.superpose(wholetraj_sliced_ecd,28)
        traj_sliced_tmd.superpose(wholetraj_sliced_tmd,28)

        reduced_cartesian_ecd.extend(ppca_ecd.transform(traj_sliced_ecd.xyz.reshape(traj_sliced_ecd.n_frames, traj_sliced_ecd.n_atoms * 3)).T[0])
        reduced_cartesian_tmd.extend(ppca_tmd.transform(traj_sliced_tmd.xyz.reshape(traj_sliced_tmd.n_frames, traj_sliced_tmd.n_atoms * 3)).T[0])
    partial_reduced_cartesian, wholetraj_sliced_ecd,wholetraj_sliced_tmd, ppca_ecd,ppca_tmd = combined_ppca_reduced_cartesian(residue_selection_1,residue_selection_2)
    reduced_cartesian_ecd = []
    reduced_cartesian_tmd = []
    for traj_note in traj_notes:
        append_projection_on_combined_ppca_data(traj_note)
    md_data['ecd_pc1']= reduced_cartesian_ecd
    md_data['tmd_pc1']= reduced_cartesian_tmd

In [95]:
def create_cppca_data_2(md_data = None,residue_selection_1 = "resSeq 13 to 198", residue_selection_2 = "resSeq 198 to 316"):
    def combined_ppca_reduced_cartesian(residue_selection_1 = "resSeq 13 to 198", residue_selection_2 = "resSeq 198 to 316"):
        wholetraj = md.load("/home/scottzhuang/masterthesis/miscellanies/pdb_ensemble/new_ensemble.pdb")
        wholetraj.superpose(wholetraj,28)
        topology = wholetraj.topology
        wholetraj_sliced_ecd = wholetraj.atom_slice(topology.select(residue_selection_1))
        wholetraj_sliced_ecd.superpose(wholetraj_sliced_ecd,28)
        wholetraj_sliced_tmd = wholetraj.atom_slice(topology.select(residue_selection_2))
        wholetraj_sliced_tmd.superpose(wholetraj_sliced_tmd,28)
        ppca_ecd = PCA(n_components=2)
        ppca_tmd = PCA(n_components=2)
        partial_reduced_cartesian = [ppca_ecd.fit_transform(wholetraj_sliced_ecd.xyz.reshape(wholetraj_sliced_ecd.n_frames, wholetraj_sliced_ecd.n_atoms * 3)), ppca_tmd.fit_transform(wholetraj_sliced_tmd.xyz.reshape(wholetraj_sliced_tmd.n_frames, wholetraj_sliced_tmd.n_atoms * 3))]    
        return partial_reduced_cartesian, wholetraj_sliced_ecd, wholetraj_sliced_tmd,ppca_ecd,ppca_tmd
    
    def append_projection_on_combined_ppca_data(traj_note, location = '/media/scottzhuang/data/MD/',skip = 10, residue_selection_1 = "resSeq 13 to 198", residue_selection_2 = "resSeq 198 to 316"):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top= location + top_location)
        topology = traj.topology
        traj_sliced_ecd = traj.atom_slice(topology.select(residue_selection_1))
        traj_sliced_tmd = traj.atom_slice(topology.select(residue_selection_2))

        traj_sliced_ecd.superpose(wholetraj_sliced_ecd,28)
        traj_sliced_tmd.superpose(wholetraj_sliced_tmd,28)

        reduced_cartesian_ecd.extend(ppca_ecd.transform(traj_sliced_ecd.xyz.reshape(traj_sliced_ecd.n_frames, traj_sliced_ecd.n_atoms * 3)).T[1])
        reduced_cartesian_tmd.extend(ppca_tmd.transform(traj_sliced_tmd.xyz.reshape(traj_sliced_tmd.n_frames, traj_sliced_tmd.n_atoms * 3)).T[1])
    partial_reduced_cartesian, wholetraj_sliced_ecd,wholetraj_sliced_tmd, ppca_ecd,ppca_tmd = combined_ppca_reduced_cartesian(residue_selection_1,residue_selection_2)
    reduced_cartesian_ecd = []
    reduced_cartesian_tmd = []
    for traj_note in traj_notes:
        append_projection_on_combined_ppca_data(traj_note)
    md_data['ecd_pc2']= reduced_cartesian_ecd
    md_data['tmd_pc2']= reduced_cartesian_tmd

In [96]:
create_cppca_data_2(md_data)

In [73]:
def create_pca_data(md_data = None):
    def ensemble_pca_cartesian():
        wholetraj = md.load("/home/scottzhuang/masterthesis/miscellanies/pdb_ensemble/new_ensemble.pdb")
        wholetraj.superpose(wholetraj,28)
        pca = PCA(n_components=2)
        reduced_cartesian = pca.fit_transform(wholetraj.xyz.reshape(wholetraj.n_frames,wholetraj.n_atoms *3))
        return wholetraj, pca
    def append_pca_data(traj_note,location = '/media/scottzhuang/data/MD/',skip = 10):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top =location + top_location)
        topology = traj.topology
        traj = traj.atom_slice(topology.select("resSeq 8 to 316"))
        traj.superpose(wholetraj,28)
        traj_reduced_cartesian = pca.transform(traj.xyz.reshape(traj.n_frames,traj.n_atoms * 3))
        print(traj_reduced_cartesian)
        pca1.extend(traj_reduced_cartesian.T[0])
        pca2.extend(traj_reduced_cartesian.T[1])
    pca1 = []
    pca2 = []
    wholetraj, pca = ensemble_pca_cartesian()
    for traj_note in traj_notes:
        append_pca_data(traj_note)
    md_data['wholepca_pc1'] = pca1
    md_data['wholepca_pc2'] = pca2

In [8]:
def create_domain_twist_data(md_data = None):
    def append_domain_twist(traj_note,location = '/media/scottzhuang/data/MD/',skip = 10):
        top_location = traj_note + '/' + traj_note + ".ca.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".ca.xtc"
        traj = md.load(location + traj_location,top= location + top_location)        
        traj.superpose(traj,0)
        topology = traj.topology
        domain_twist = []
        for chain in range (0,5):
            residue_selection_1 = "resid " + str(8+chain*311) + " to " + str(192+chain*311)
            residue_selection_2 = "resid " + str(192+chain*311) + " to " + str(310+chain*311)
            traj_sliced_ecd = traj.atom_slice(topology.select(residue_selection_1))
            traj_sliced_tmd = traj.atom_slice(topology.select(residue_selection_2))
            angle = []
            for i in range(0,traj.n_frames):
                cen_mass_ecd = md.compute_center_of_mass(traj_sliced_ecd[i])[0]
                cen_mass_tmd = md.compute_center_of_mass(traj_sliced_tmd[i])[0]
                cen_mass = md.compute_center_of_mass(traj[i])[0]
                cen_mass_ecd[2] = cen_mass[2]
                cen_mass_tmd[2] = cen_mass[2]
                vec_ecd = cen_mass_ecd - cen_mass
                vec_tmd = cen_mass_tmd - cen_mass
                veclength_ecd = np.sqrt(np.sum(np.power(vec_ecd,2)))
                veclength_tmd = np.sqrt(np.sum(np.power(vec_tmd,2)))
                angle.append(57.2958 * np.arccos(np.dot(vec_ecd,vec_tmd) /(veclength_ecd * veclength_tmd)))
            domain_twist.append(angle)
        domain_twist_data.extend(np.mean(np.asarray(domain_twist),axis=0))
    domain_twist_data = []
    for traj_note in traj_notes:
        domain_twist_avg = append_domain_twist(traj_note)
    md_data['domain twist']= domain_twist_data

In [22]:
def create_hole_data(md_data = None):
    import MDAnalysis as mda
    from MDAnalysis.analysis.hole import HOLEtraj
    def append_hole_data(traj_note,location = '/media/scottzhuang/data/MD/',skip = 10): 
        hole_traj_location = location + traj_note + '/' + traj_note + ".hole.traj.pdb"
        u = mda.Universe(hole_traj_location)
        H = HOLEtraj(u, executable="~/hole2/exe/hole")
        H.run()
        hole_data.extend(H.min_radius().T[1])
    hole_data = []
    for traj_note in traj_notes:
        append_hole_data(traj_note)
    md_data['min hole radius']=hole_data

In [47]:
def create_hbond_data(md_data = None):
    import MDAnalysis as mda
    from MDAnalysis.analysis.hbonds import HydrogenBondAnalysis
    def append_hbond_data(traj_note,location = '/media/scottzhuang/data/MD/', skip=10):
        top_location = traj_note + '/' + traj_note + ".protein.gro"
        traj_location = traj_note + '/' + traj_note + ".skip" + str(skip) + ".protein.xtc"
        traj = mda.Universe(location + top_location,location + traj_location)
        h = HydrogenBondAnalysis(traj,'protein','protein',distance=3.0, angle=120.0)
        h.run()
        hbond_data.extend(h.count_by_time().T[1])
    hbond_data = []
    for traj_note in traj_notes:
        append_hbond_data(traj_note)
    md_data['h_bond number'] = hbond_data

In [91]:
#md_data = create_md_dataframe()
#create_metadata(md_data= md_data)
md_data = pd.read_csv("glic_gating.csv")

In [11]:
md_data = create_md_dataframe()
create_metadata(md_data= md_data)
create_system_notation(md_data = md_data)
create_rmsd_data(md_data= md_data)
create_pca_data(md_data = md_data)
create_cppca_data(md_data= md_data)
create_domain_twist_data(md_data= md_data)
create_hole_data(md_data= md_data)
#create_hbond_data(md_data=md_data)

In [99]:
md_data.to_csv("glic_gating.csv")

In [97]:
md_data.columns

Index(['Unnamed: 0', 'MD_name', 'pH', 'replicate', 'traj_time', 'rmsd',
       'ecd_pc1', 'tmd_pc1', 'domain twist', 'min hole radius', 'system',
       'wholepca_pc1', 'wholepca_pc2', 'rmsf', 'ecd_pc2', 'tmd_pc2'],
      dtype='object')

In [98]:
md_data

,Unnamed: 0,MD_name,pH,replicate,traj_time,rmsd,ecd_pc1,tmd_pc1,domain twist,min hole radius,system,wholepca_pc1,wholepca_pc2,rmsf,ecd_pc2,tmd_pc2
0,0,5NJY,70,1,0,0.000000,-0.450258,3.279069,11.755106,0.83748,0,0.717673,3.350286,"[ 0.3902517 0.316796 0.20989354 ..., 0.1...",0.357732,-1.119838
1,1,5NJY,70,1,1,1.727738,0.894403,3.166664,9.105226,0.73840,0,0.630748,4.148447,"[ 0.28054386 0.26561692 0.20901562 ..., 0.0...",2.561647,-1.134746
2,2,5NJY,70,1,2,2.020854,2.341526,2.749989,9.638441,1.29015,0,1.855654,3.210543,"[ 0.07622263 0.07348195 0.09509178 ..., 0.1...",2.918209,-1.380760
3,3,5NJY,70,1,3,1.915493,1.985111,3.009500,8.880064,1.41160,0,1.412526,3.407093,"[ 0.1024719 0.16225648 0.18653138 ..., 0.1...",1.983271,-1.233216
4,4,5NJY,70,1,4,1.899955,2.116413,3.058160,12.076026,0.94805,0,2.364888,3.248358,"[ 0.14206782 0.08395956 0.09712773 ..., 0.1...",2.501530,-1.161252
5,5,5NJY,70,1,5,2.043537,2.543423,2.773529,9.824443,1.44453,0,1.866260,2.947303,"[ 0.14614813 0.17722042 0.21123332 ..., 0.1...",1.713744,-1.081816
6,6,5NJY,70,1,6,1.885621,2.745998,2.625010,10.523686,1.52369,0,2.240279,2.618280,"[ 0.16406532 0.16484803 0.1436452 ..., 0.1...",1.789053,-1.059884
7,7,5NJY,70,1,7,1.995083,3.198771,2.811840,10.740198,1.42044,0,2.609150,2.053408,"[ 0.19980372 0.12892586 0.07407645 ..., 0.0...",0.104684,-1.109357
8,8,5NJY,70,1,8,2.042677,3.669586,2.774023,12.822112,1.04020,0,3.700863,2.066898,"[ 0.16702205 0.19065368 0.19865094 ..., 0.2...",1.607017,-1.246203
9,9,5NJY,70,1,9,2.152671,3.925136,3.081735,11.511422,1.72425,0,3.595905,2.555981,"[ 0.10299914 0.15221907 0.15970014 ..., 0.2...",1.711967,-1.297545
